# Running Oktoberfest

## 1- Import necessary python packages

In [1]:
import os
from oktoberfest.runner import run_job
import json
import urllib.request
import shutil
from tqdm import tqdm
%load_ext autoreload
%autoreload 2

## 2- Download files from zenodo required to run different tasks

### A- Get the current directory and set the file name

In [2]:
download_dir = os.getcwd()
download_file = os.path.join(download_dir, 'Oktoberfest_input.zip')
url = 'https://zenodo.org/record/7613029/files/Oktoberfest_input.zip'

download = False  # set this to false if you already have the file and don't want to download again in the next step

### B- Download and extract files from zenodo to the same directory

In [3]:
if download:
    with tqdm(unit="B", total=2739196307, unit_scale=True, unit_divisor=1024, miniters=1, desc=url.split("/")[-1]) as t:
        urllib.request.urlretrieve(url=url, filename=download_file, reporthook=lambda blocks, block_size, _: t.update(blocks * block_size - t.n))
    shutil.unpack_archive(download_file, download_dir)

### C- Check downloaded files

In [4]:
input_dir = '/home/karim/projects/oktoberfest/tutorials/Oktoberfest_input'
print(f'Downloaded data is stored in {input_dir}\nContents:')
os.listdir(input_dir)

Downloaded data is stored in /home/karim/projects/oktoberfest/tutorials/Oktoberfest_input
Contents:


['config_files',
 'GN20170722_SK_HLA_G0103_R1_02.raw',
 'results.sage.tsv',
 'GN20170722_SK_HLA_G0103_R2_01.raw',
 'GN20170722_SK_HLA_G0103_R2_02.raw',
 'GN20170722_SK_HLA_G0103_R1_01.raw',
 'peptides_spectral_library.csv']

## 3- Running Different Tasks

### A- Spectral Library Generation

#### Generate config file

In [5]:
task_config_spectral_lib = {
    "type": "SpectralLibraryGeneration",
    "tag": "",
    "inputs": {
            "search_results": input_dir + "/results.sage.tsv",
            "search_results_type": "Sage",
            "library_input": input_dir + "/peptides_spectral_library.csv",
            "library_input_type": "peptides"
        },
    "output": "./out",
    "models":{
        "intensity": "Prosit_2020_intensity_HCD",
        "irt": "Prosit_2019_irt"
    },
    "outputFormat": "spectronaut",
    "prediction_server": "koina.proteomicsdb.org:443",
    "ssl": True,
}

#### Save config as json

In [6]:
with open('./spectral_library_config.json', 'w') as fp:
    json.dump(task_config_spectral_lib, fp)

#### Run spectral library generation job

In [7]:
run_job("./spectral_library_config.json")

2023-11-10 12:31:51,740 - INFO - oktoberfest.utils.config::read Reading configuration from ./spectral_library_config.json
2023-11-10 12:31:51,741 - INFO - oktoberfest.runner::run_job Oktoberfest version 0.5.1
Copyright 2023, Wilhelmlab at Technical University of Munich
2023-11-10 12:31:51,742 - INFO - oktoberfest.runner::run_job Job executed with the following config:
2023-11-10 12:31:51,742 - INFO - oktoberfest.runner::run_job {
    "type": "SpectralLibraryGeneration",
    "tag": "",
    "inputs": {
        "search_results": "/home/karim/projects/oktoberfest/tutorials/Oktoberfest_input/results.sage.tsv",
        "search_results_type": "Sage",
        "library_input": "/home/karim/projects/oktoberfest/tutorials/Oktoberfest_input/peptides_spectral_library.csv",
        "library_input_type": "peptides"
    },
    "output": "./out",
    "models": {
        "intensity": "Prosit_2020_intensity_HCD",
        "irt": "Prosit_2019_irt"
    },
    "outputFormat": "spectronaut",
    "prediction_s

Inferring predictions for 5 spectra with batch site 1000:   0%|          | 0/1 [00:00<?, ?it/s]

Inferring predictions for 5 spectra with batch site 1000:   0%|          | 0/1 [00:00<?, ?it/s]

### B- CE Calibration

#### Generate config file

In [8]:
task_config_ce_calibration = {
    "type": "CollisionEnergyCalibration",
    "tag": "",
    "inputs":{
        "search_results": input_dir + "/results.sage.tsv",
        "search_results_type": "Sage",
        "spectra": input_dir,
        "spectra_type": "raw"
    },
    "output": "./out",
    "models": {
        "intensity": "Prosit_2020_intensity_HCD",
        "irt": "Prosit_2019_irt"
    },
    "prediction_server": "koina.proteomicsdb.org:443",
    "ssl": True,
    "thermoExe": "/home/karim/non_git/essentials/therm/ThermoRawFileParser.exe",
    "massTolerance": 20,
    "unitMassTolerance": "ppm",
    "numThreads": 1
}

#### Save config as json

In [9]:
with open('./ce_calibration_config.json', 'w') as fp:
    json.dump(task_config_ce_calibration, fp)

#### Run ce calibration job

In [ ]:
run_job("./ce_calibration_config.json")

2023-11-10 12:31:54,211 - INFO - oktoberfest.utils.config::read Reading configuration from ./ce_calibration_config.json
2023-11-10 12:31:54,212 - INFO - oktoberfest.runner::run_job Oktoberfest version 0.5.1
Copyright 2023, Wilhelmlab at Technical University of Munich
2023-11-10 12:31:54,213 - INFO - oktoberfest.runner::run_job Job executed with the following config:
2023-11-10 12:31:54,213 - INFO - oktoberfest.runner::run_job {
    "type": "CollisionEnergyCalibration",
    "tag": "",
    "inputs": {
        "search_results": "/home/karim/projects/oktoberfest/tutorials/Oktoberfest_input/results.sage.tsv",
        "search_results_type": "Sage",
        "spectra": "/home/karim/projects/oktoberfest/tutorials/Oktoberfest_input",
        "spectra_type": "raw"
    },
    "output": "./out",
    "models": {
        "intensity": "Prosit_2020_intensity_HCD",
        "irt": "Prosit_2019_irt"
    },
    "prediction_server": "koina.proteomicsdb.org:443",
    "ssl": true,
    "thermoExe": "/home/kari

Waiting for tasks to complete:   0%|          | 0/1 [00:00<?, ?it/s]

2023-11-10 12:31:54,418 - INFO - spectrum_io.file.hdf5::read_file Reading sparse matrix from hdf5 file. Available keys: <KeysViewHDF5 ['meta_data', 'sparse_raw_intensity', 'sparse_raw_mz']>
2023-11-10 12:31:54,451 - INFO - spectrum_io.file.hdf5::read_file Reading sparse matrix from hdf5 file. Available keys: <KeysViewHDF5 ['meta_data', 'sparse_raw_intensity', 'sparse_raw_mz']>


Inferring predictions for 32000 spectra with batch site 1000:   0%|          | 0/32 [00:00<?, ?it/s]

Exception in thread Thread-8 (_handle_results):
Traceback (most recent call last):
  File "/home/karim/miniconda3/envs/oktoberfest/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/home/karim/miniconda3/envs/oktoberfest/lib/python3.10/threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "/home/karim/miniconda3/envs/oktoberfest/lib/python3.10/multiprocessing/pool.py", line 576, in _handle_results
    task = get()
  File "/home/karim/miniconda3/envs/oktoberfest/lib/python3.10/multiprocessing/connection.py", line 256, in recv
    return _ForkingPickler.loads(buf.getbuffer())
TypeError: InferenceServerException.__init__() missing 1 required positional argument: 'msg'


### C- Rescoring

#### Generate config file

In [20]:
task_config_rescoring = {
    "type": "Rescoring",
    "tag": "",
    "inputs":{
        "search_results": input_dir + "/results.sage.tsv",
        "search_results_type": "Sage",
        "spectra": input_dir,
        "spectra_type": "raw"
    },
    "output": "./out",
    "models": {
        "intensity": "Prosit_2020_intensity_HCD",
        "irt": "Prosit_2019_irt"
    },
    "prediction_server": "koina.proteomicsdb.org:443",
    "ssl": True,
    "thermoExe": "ThermoRawFileParser.exe",
    "numThreads": 4,
    "fdr_estimation_method": "percolator",
    "regressionMethod": "spline",
    "allFeatures": False,
    "massTolerance": 20,
    "unitMassTolerance": "ppm"
}

#### Save config as json

In [21]:
with open('./rescoring_config.json', 'w') as fp:
    json.dump(task_config_rescoring, fp)

#### Run rescoring job

In [22]:
run_job("rescoring_config.json")

2023-11-09 14:12:19,263 - INFO - oktoberfest.utils.config::read Reading configuration from rescoring_config.json
2023-11-09 14:12:19,264 - INFO - oktoberfest.runner::run_job Oktoberfest version 0.5.1
Copyright 2023, Wilhelmlab at Technical University of Munich
2023-11-09 14:12:19,265 - INFO - oktoberfest.runner::run_job Job executed with the following config:
2023-11-09 14:12:19,265 - INFO - oktoberfest.runner::run_job {
    "type": "Rescoring",
    "tag": "",
    "inputs": {
        "search_results": "/home/karim/non_git/oktoberfest_runs/default_zenodo/raw_zenodo_mq/results.sage.tsv",
        "search_results_type": "Sage",
        "spectra": "/home/karim/non_git/oktoberfest_runs/default_zenodo/raw_zenodo_mq",
        "spectra_type": "raw"
    },
    "output": "./out",
    "models": {
        "intensity": "Prosit_2020_intensity_HCD",
        "irt": "Prosit_2019_irt"
    },
    "prediction_server": "koina.proteomicsdb.org:443",
    "ssl": true,
    "thermoExe": "ThermoRawFileParser.exe"

Waiting for tasks to complete:   0%|          | 0/4 [00:00<?, ?it/s]

2023-11-09 14:12:19,317 - INFO - oktoberfest.utils.process_step::is_done Skipping ce_calib.GN20170722_SK_HLA_G0103_R1_01 step because out/proc/ce_calib.GN20170722_SK_HLA_G0103_R1_01.done was found.
2023-11-09 14:12:19,317 - INFO - oktoberfest.utils.process_step::is_done Skipping ce_calib.GN20170722_SK_HLA_G0103_R2_02 step because out/proc/ce_calib.GN20170722_SK_HLA_G0103_R2_02.done was found.
2023-11-09 14:12:19,317 - INFO - oktoberfest.utils.process_step::is_done Skipping ce_calib.GN20170722_SK_HLA_G0103_R2_01 step because out/proc/ce_calib.GN20170722_SK_HLA_G0103_R2_01.done was found.
2023-11-09 14:12:19,375 - INFO - spectrum_io.file.hdf5::read_file Reading sparse matrix from hdf5 file. Available keys: <KeysViewHDF5 ['meta_data', 'sparse_pred_intensity', 'sparse_raw_intensity', 'sparse_raw_mz']>
2023-11-09 14:12:19,376 - INFO - spectrum_io.file.hdf5::read_file Reading sparse matrix from hdf5 file. Available keys: <KeysViewHDF5 ['meta_data', 'sparse_pred_intensity', 'sparse_raw_intens

Inferring predictions for 563 spectra with batch site 1000:   0%|          | 0/1 [00:00<?, ?it/s]

Inferring predictions for 1404 spectra with batch site 1000:   0%|          | 0/2 [00:00<?, ?it/s]

...

2023-11-09 13:12:24,420 - ERROR - spectrum_fundamentals.metrics.percolator::get_indices_below_fdr Could not find any targets below 0.10999999999999999 out of 707 targets in total


2023-11-09 13:12:24,420 - ERROR - spectrum_fundamentals.metrics.percolator::get_indices_below_fdr Could not find any targets below 0.10999999999999999 out of 707 targets in total


2023-11-09 13:12:24,429 - INFO - spectrum_fundamentals.metrics.percolator::apply_lda_and_get_indices_below_fdr Found 0 targets and 697 decoys as input for the LDA model
2023-11-09 13:12:24,463 - ERROR - spectrum_fundamentals.metrics.percolator::get_indices_below_fdr Could not find any targets below 0.10999999999999999 out of 707 targets in total


2023-11-09 13:12:24,463 - ERROR - spectrum_fundamentals.metrics.percolator::get_indices_below_fdr Could not find any targets below 0.10999999999999999 out of 707 targets in total
/home/karim/projects/spectrum_fundamentals/spectrum_fundamentals/metrics/percolator.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retention_time_df["rt_bin_index"] = np.digitize(retention_time_df["RETENTION_TIME"], break_points)
/home/karim/miniconda3/envs/oktoberfest/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/karim/miniconda3/envs/oktoberfest/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar div

2023-11-09 13:12:24,551 - INFO - spectrum_fundamentals.metrics.percolator::calc Median absolute error predicted vs observed retention time on targets < 1% FDR: nan
2023-11-09 14:12:24,636 - INFO - oktoberfest.runner::run_rescoring Merging input tab files for rescoring without peptide property prediction
2023-11-09 14:12:24,674 - INFO - oktoberfest.runner::run_rescoring Merging input tab files for rescoring with peptide property prediction
2023-11-09 14:12:24,814 - INFO - oktoberfest.rescore.rescore::rescore_with_percolator Starting percolator with command percolator --weights out/results/percolator/original.percolator.weights.csv                     --num-threads 3                     --subset-max-train 500000                     --post-processing-tdc                     --testFDR 0.01                     --trainFDR 0.01                     --results-psms out/results/percolator/original.percolator.psms.txt                     --decoy-results-psms out/results/percolator/original.percola